In [ ]:
def resample_poly(gdf, distance): # gdf contains a single polygon
    poly = gdf.geometry[0]    
    exterior_coords = list(poly.exterior.coords)
    exterior_line = LineString(exterior_coords)
    resampled_line = []
    current_distance = 0
    while current_distance <= exterior_line.length:
        resampled_line.append(exterior_line.interpolate(current_distance))
        current_distance += distance
    coords = []
    for point in resampled_line:
        x,y = point.x, point.y
        coords.append((x,y))
    poly_resampled = Polygon(coords) # Keep polygons as Shapely polygons
    return(poly_resampled) # Returns a Shapely polygon

def resample_linestring(linestring, distance):
    total_length = linestring.length
    num_points = int(total_length / distance)
    points = [linestring.interpolate(distance * i) for i in range(num_points + 1)]
    return points

In [ ]:
# Preparing meshes for boundaries, bores and fault. 

def prepboundarymesh(P, grid): # MODEL BOUNDARY
    
    x0, x1, y0, y1 = P.x0, P.x1, P.y0, P.y1 
    w, r  = P.w, P.r
    Lx, Ly = x1 - x0, y1 - y0
    
    if grid == 'tri':
        model_vertices = []
        for i in range(r): model_vertices.append(((i/r * Lx)+x0,y0))         # Bottom
        for i in range(r): model_vertices.append((x1, (i/r * Ly) + y0))      # Right
        for i in range(r): model_vertices.append(((Lx - i/r * Lx) + x0, y1)) # Top
        for i in range(r): model_vertices.append((x0, (Ly - i/r * Ly) +y0))  # Left

        # INTERIOR BOUNDARY
        interior_vertices = [(x0+w,y0+w),(x1-w,y0+w),(x1-w, y1-w),(x0+w,y1-w)]
        interior_poly = Polygon(interior_vertices)
        
    if grid == 'vor':
        model_vertices = []
        for i in range(r): model_vertices.append(((i/r * Lx)+x0,y0))         # Bottom
        for i in range(r): model_vertices.append((x1, (i/r * Ly) + y0))      # Right
        for i in range(r): model_vertices.append(((Lx - i/r * Lx) + x0, y1)) # Top
        for i in range(r): model_vertices.append((x0, (Ly - i/r * Ly) +y0))  # Left

        # INTERIOR BOUNDARY
        interior_vertices = [(x0+w,y0+w),(x1-w,y0+w),(x1-w, y1-w),(x0+w,y1-w)]
        interior_poly = Polygon(interior_vertices)
    
    return(model_vertices, interior_vertices)

def prepboremesh(P, grid):
    
    theta = np.linspace(0, 2 * np.pi, 11)

    pump_bores_inner, pump_bores_outer = [], []
    obs_bores_inner, obs_bores_outer = [], []
    
    if grid == 'tri':
        
        def vertices_equtri1(X, Y, l): # l is distance from centre of triangle to vertex
            x1 = X - l*3**0.5/2 
            x2 = X + l*3**0.5/2 
            x3 = X
            y1 = Y - l/2
            y2 = Y - l/2
            y3 = Y + l
            return(x1, x2, x3, y1, y2, y3)
        
        def vertices_equtri2(X, Y, l): # l is distance from centre of triangle to vertex
            x1 = X 
            x2 = X + l*3**0.5
            x3 = X - l*3**0.5
            y1 = Y - 2*l
            y2 = Y + l
            y3 = Y + l
            return(x1, x2, x3, y1, y2, y3)

        for i in P.xypumpbores:   
            X, Y = i[0], i[1] # coord of pumping bore
                        
            x1, x2, x3, y1, y2, y3 = vertices_equtri1(X, Y, P.radius1/2)
            vertices_inner = ((x1, y1), (x2, y2), (x3, y3))
            x1, x2, x3, y1, y2, y3 = vertices_equtri2(X, Y, P.radius1/2)
            vertices_outer = ((x1, y1), (x2, y2), (x3, y3))
            
            pump_bores_inner.append(vertices_inner)
            pump_bores_outer.append(vertices_outer)
            
        for i in P.xyobsbores:   
            X, Y = i[0], i[1] # coord of pumping bore
                        
            x1, x2, x3, y1, y2, y3 = vertices_equtri1(X, Y, P.radius1/2)
            vertices_inner = ((x1, y1), (x2, y2), (x3, y3))
            x1, x2, x3, y1, y2, y3 = vertices_equtri2(X, Y, P.radius1/2)
            vertices_outer = ((x1, y1), (x2, y2), (x3, y3))
            
            obs_bores_inner.append(vertices_inner)
            obs_bores_outer.append(vertices_outer)
            
        return(pump_bores_inner, pump_bores_outer, obs_bores_inner, obs_bores_outer)
    
    if grid == 'vor':
        for i in P.xypumpbores:   
            X = i[0] + P.radius1 * np.cos(theta)
            Y = i[1] + P.radius1 * np.sin(theta)    
            vertices_inner = [(x_val, y_val) for x_val, y_val in zip(X, Y)]
            X = i[0] + P.radius2 * np.cos(theta)
            Y = i[1] + P.radius2 * np.sin(theta)    
            vertices_outer = [(x_val, y_val) for x_val, y_val in zip(X, Y)]
            pump_bores_inner.append(vertices_inner)
            pump_bores_outer.append(vertices_outer)
            
        for i in P.xyobsbores:   
            X = i[0] + P.radius1 * np.cos(theta)
            Y = i[1] + P.radius1 * np.sin(theta)    
            vertices_inner = [(x_val, y_val) for x_val, y_val in zip(X, Y)]
            X = i[0] + P.radius2 * np.cos(theta)
            Y = i[1] + P.radius2 * np.sin(theta)    
            vertices_outer = [(x_val, y_val) for x_val, y_val in zip(X, Y)]
            obs_bores_inner.append(vertices_inner)
            obs_bores_outer.append(vertices_outer)

        return(pump_bores_inner, pump_bores_outer, obs_bores_inner, obs_bores_outer)

def prepare_fault_nodes_voronoi(P, shpfilepath, model_boundary):
    # Import fault and turn into a linestring
    #gdf = gpd.read_file('../shp/badaminna_fault.shp') 
    gdf = gpd.read_file(shpfilepath) 
    fault = gpd.clip(gdf, model_boundary) # fault is a gdf
    df = fault.get_coordinates()
    fault_points = list(zip(list(df.x), list(df.y)))
    fault_linestring = LineString(fault_points)

    # Settings to make point cloud
    L = P.fault_buffer
    Lfault = fault.length
    r = 2*L/3 # distance between points

    # Fault point cloud
    offsets = [-1.5*r, -0.5*r, 0.5*r, 1.5*r]
    fault_offset_lines = []
    for offset in offsets:
        ls = fault_linestring.parallel_offset(offset) # linestring.parallel_offset
        ls_resample = resample_linestring(ls, r)
        p = []
        for point in ls_resample:
            if inner_boundary.contains(point):
                x,y = point.x, point.y
                p.append((x,y))
        offset_ls = LineString(p)
        coords = list(offset_ls.coords)
        fault_offset_lines.append(coords)

    fault_refinement_nodes = [tup for line in fault_offset_lines for tup in line]
    
    return(fault_refinement_nodes)
    
def prepfaultmesh(P, grid): # This has been made for a pretend fault. Will look different for a real one!
    
    L = P.fault_buffer
    Lfault = np.sqrt((P.fx2-P.fx1)**2+(P.fy2 - P.fy1)**2)
    from shapely.geometry import LineString,Point,Polygon,shape
         # refining factor - adds points along fault for triangulation
    fs = (P.fy2 - P.fy1)/(P.fx2 - P.fx1) # fault strike in xy
    fp = -1/fs # direction perpendiculr to fault strike
    theta = math.atan(fp)
    phi = math.pi/2 - theta
    
    K = P.fault_buffer
    fp1 = (P.fx1 + K * np.cos(phi), P.fy1 + K * np.sin(phi))
    fp2 = (P.fx2 + K * np.cos(phi), P.fy2 + K * np.sin(phi))
    fp3 = (P.fx2 - K * np.cos(phi), P.fy2 - K * np.sin(phi))
    fp4 = (P.fx1 - K * np.cos(phi), P.fy1 - K * np.sin(phi))
    P.fault_poly = Polygon((fp1, fp2, fp3, fp4))
    
    if grid == 'tri':
        
        # THESE WILL BE NODES
        r = 50      # refining factor
        fault_points = []
        for i in range(r+1): 
            fault_points.append((P.fx1 + i * (P.fx2-P.fx1)/r , P.fy1 + i * (P.fy2-P.fy1)/r)) 
    
    if grid == 'vor':

        r = 2*L/3 # distance between points

        x_array = np.arange(0, Lfault, r)  # Create a cloud of points to refine around fault
        y_array = np.arange(-L, L + r, r)

        fault_points = []
        for i in range(len(x_array)):# vertical points 
            for j in range(len(y_array)): # horizontal points
                x = x_array[i] * np.cos(phi) + y_array[j] * np.sin(phi)
                y = x_array[i] * -np.sin(phi) + y_array[j] * np.cos(phi)
                fault_points.append((P.fx1 + x, P.fy1 + y))
            
    return(fault_points)

In [ ]:
# PREPARING NODES AND POLYGONS AND THEN CALLING MESHING FUNCTION

def createcell2d(P, grid, fault = True):
    
    if grid == 'car':
        delr = P.delx * np.ones(P.ncol, dtype=float)
        delc = P.dely * np.ones(P.nrow, dtype=float)
        top  = np.ones((P.nrow, P.ncol), dtype=float)
        botm = np.zeros((1, P.nrow, P.ncol), dtype=float)

        sg = flopy.discretization.StructuredGrid(delr=delr, delc=delc, top=top, botm=botm)
        xycenters = sg.xycenters
        
        cell2d = []
        xcyc = [] # added 
        for n in range(P.nrow*P.ncol):
            l,r,c = sg.get_lrc(n)[0]
            xc = xycenters[0][c]
            yc = xycenters[1][r]
            iv1 = c + r * (P.ncol + 1)  # upper left
            iv2 = iv1 + 1
            iv3 = iv2 + P.ncol + 1
            iv4 = iv3 - 1
            cell2d.append([n, xc, yc, 5, iv1, iv2, iv3, iv4, iv1])
            xcyc.append((xc, yc))
        
        vertices = []
        xa = np.arange(P.x0, P.x1 + P.delx, P.delx)      
        ya = np.arange(P.y1, P.y0 - P.dely/2, -P.dely)

        n = 0
        for j in ya:
            for i in xa:
                vertices.append([n, i, j])
                n+=1
                
        return(cell2d, xcyc, vertices, sg)
    
    if grid == 'tri': 
        boresinner, boresouter, obsinner, obsouter = prepboremesh(P, grid = grid)
        modelextpoly, modelintpoly = prepboundarymesh(P, grid = grid)
            
        nodes = []
        for bore in boresinner: 
            for n in bore: nodes.append(n)
        for bore in boresouter: 
            for n in bore: nodes.append(n)
        for bore in obsinner: 
            for n in bore: nodes.append(n)
        for bore in obsouter: 
            for n in bore: nodes.append(n)
        if fault == True:
            faultpoints = prepfaultmesh(P, grid = grid)
            for point in faultpoints:
                nodes.append(point)
        if fault == False:
            del P.fault_poly
        
        nodes = np.array(nodes)
        
        polygons = []
        polygons.append((modelextpoly, (P.x0 + 10, P.y0 + 10), P.boundmaxtri)) # Inside boundary frame
        polygons.append((modelintpoly, (P.x0 + P.w + 10, P.y0 + P.w + 10), P.modelmaxtri)) # Bulk of model!       
        
        cell2d, xcyc, vertices, gridobject = tri_meshing(P, polygons, nodes)
        
        return(cell2d, xcyc, vertices, gridobject, nodes)
        
    if grid == 'vor': 
        pumpinner, pumpouter, obsinner, obsouter = prepboremesh(P, grid = grid)
        modelextpoly, modelintpoly = prepboundarymesh(P, grid = grid)
        faultpoints = prepfaultmesh(P, grid = grid)
        
        nodes = []
        #for point in modelextpoly: # Added back 29/4
        #    nodes.append(point) # Added back 29/4
        #for point in modelintpoly: # Added back 29/4
        #    nodes.append(point) # Added back 29/4
        for point in P.xypumpbores:
            nodes.append(point)
        for point in P.xyobsbores:
            nodes.append(point)
        if fault == True:
            for point in faultpoints:
                nodes.append(point)
        if fault == False:
            del P.fault_poly

        nodes = np.array(nodes)
        
        polygons = []
        polygons.append((modelextpoly, (P.x0 + 10, P.y0 + 10), P.boundmaxtri)) # Inside boundary frame
        polygons.append((modelintpoly, (P.x0 + P.w + 10, P.y0 + P.w + 10), P.modelmaxtri)) # Bulk of model!     
        
        for i in range(P.npump): # Append pumping bore zone polygons
            polygons.append((pumpinner[i], P.xypumpbores[i], P.boremaxtri))
            polygons.append((pumpouter[i],0, 0)) # 0, 0 means don't refine inside polygon
            
        for i in range(P.nobs): # Append pumping bore zone polygons
            polygons.append((obsinner[i], P.xyobsbores[i], P.boremaxtri))
            #polygons.append((obsouter[i],0, 0)) # 0, 0 means don't refine inside polygon
        
        cell2d, xcyc, vertices, gridobject = vor_meshing(P, polygons, nodes)
    
        return(cell2d, xcyc, vertices, gridobject, nodes)


In [ ]:
#BUILDING TRI AND VORONOI MESH

def tri_meshing(P, polygons, nodes):

    import flopy
    from flopy.discretization import VertexGrid
    from flopy.utils.triangle import Triangle as Triangle
    
    tri = Triangle(angle=P.angle, model_ws=P.workspace, exe_name=P.triExeName, nodes = nodes,
                   additional_args = ['-j','-D'])

    for poly in polygons:
        tri.add_polygon(poly[0]) 
        if poly[1] != 0: # Flag set to zero if region not required
            tri.add_region(poly[1], 0, maximum_area = poly[2]) # Picks a point in main model

    tri.build(verbose=False) # Builds triangular grid

    cell2d = tri.get_cell2d()     # cell info: id,x,y,nc,c1,c2,c3 (list of lists)
    vertices = tri.get_vertices()
    xcyc = tri.get_xcyc()
    
    return(cell2d, xcyc, vertices, tri)

def vor_meshing(P, polygons, nodes):

    import flopy
    from flopy.discretization import VertexGrid
    from flopy.utils.triangle import Triangle as Triangle
    from flopy.utils.voronoi import VoronoiGrid
    
    tri = Triangle(angle=P.angle, model_ws=P.workspace, exe_name=P.triExeName, nodes = nodes,
                   additional_args = ['-j','-D'])

    for poly in polygons:
        tri.add_polygon(poly[0]) 
        if poly[1] != 0: # Flag set to zero if region not required
            tri.add_region(poly[1], 0, maximum_area = poly[2]) # Picks a point in main model

    tri.build(verbose=False) # Builds triangular grid

    vor = VoronoiGrid(tri)
    vertices = vor.get_disv_gridprops()['vertices']
    cell2d = vor.get_disv_gridprops()['cell2d']

    xcyc = []
    for cell in cell2d:
        xcyc.append((cell[1],cell[2]))
    
    return(cell2d, xcyc, vertices, vor)

In [ ]:
def plot_cell2d_car(P, xlim, ylim):
    
    fig = plt.figure(figsize=(7,7))
    ax = plt.subplot(1, 1, 1, aspect='equal')
    lc = P.sg.plot(color = 'gray', lw = 0.5) 
    for i in P.xcyccar:
        plt.plot(i[0],i[1], marker='o', markersize = '0.5', color='red')
    for i in range(P.npump):
        ax.plot(P.xypumpbores[i], ms = 5, color = 'black')
    ax.plot((P.fx1, P.fx2), (P.fy1, P.fy2), color = 'purple', lw = 1)  #fault
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    ax.set_title('Nodes in 2D: ' + str(len(P.cell2dcar)))

def plot_cell2d_tri(P, xlim, ylim):
    
    fig = plt.figure(figsize=(7,7))
    ax = plt.subplot(1, 1, 1, aspect='equal')
    P.tri.plot(edgecolor='gray', lw = 0.5)
    P.tri.plot_centroids(ax=ax, marker='o', markersize = '0.5', color='red')
    numberBoundaries = P.tri.get_boundary_marker_array().max()+1
    cmap = plt.colormaps["hsv"]
    labelList = list(range(1,numberBoundaries))
    i = 0
    for ibm in range(1,numberBoundaries):
        P.tri.plot_boundary(ibm=ibm, ax=ax,marker='o', ms = 0.2, color=cmap(ibm), label= ibm)  
    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = OrderedDict(zip(labels, handles))
    for i in range(P.npump):
        ax.plot(P.xypumpbores[i], ms = 5, color = 'black')
    for i in trinodes: 
        ax.plot(i[0], i[1], 'o', ms = 2, color = 'black')
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    ax.set_title('Nodes in 2D: ' + str(len(P.cell2dtri)))
    
def plot_cell2d_vor(P, xlim, ylim): #xlim = [x0, x1], ylim = [y0, y1]):   
    fig = plt.figure(figsize=(7,7))
    ax = plt.subplot(1, 1, 1, aspect='equal')
    P.vor.plot(edgecolor='black', lw = 0.4)
    for i in P.xcycvor: ax.plot(i[0], i[1], 'o', color = 'green', ms = 1.5)
    for i in range(P.npump):
        ax.plot(P.xypumpbores[i], ms = 2, color = 'black')
    for i in vornodes: ax.plot(i[0], i[1], 'o', ms = 2, color = 'black')
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    ax.set_title('Nodes in 2D: ' + str(len(P.cell2dvor)))

In [ ]:
print('Meshing routines loaded!')